<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
Only Submission(LoadLocalTrainModel)
</b></h1> 

<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
Inference Pipeline
</b></h1> 

## **》》》 [IMPORTANT] Env Params**

In [ ]:
""" Train Model """
model_path = "/kaggle/input/yolo11n-byu/yolo/yolo11x_fold1.pt"

""" [IMPORTANT]
* This parameter has a significant impact on the value of LB since it is the threshold for the prediction score inferred by the model.
* In my experiments, 0.5 to 0.55 is optimal for local CV, but when submitting, 0.35 to 0.45 seems to give better results, so there is a difference.
"""
YOLO_THR = 0.05
GNN_THR = 0.05

MAX_DETECTIONS_PER_TOMO = 3
NMS_IOU_THRESHOLD = 0.2
CONCENTRATION = 1
BATCH_SIZE = 8 

In [ ]:
knn_folds = [
    '/kaggle/input/yolo11x-byu/checkpoint/knn_graph_12_8_4.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/knn_graph_12_8_3.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/knn_graph_12_8_2.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/knn_graph_12_8_1.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/knn_graph_12_8_0.pth'
]

delaunay_folds = [
    '/kaggle/input/yolo11x-byu/checkpoint/delaunay_graph_15_8_4.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/delaunay_graph_15_8_3.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/delaunay_graph_15_8_2.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/delaunay_graph_15_8_1.pth',
    '/kaggle/input/yolo11x-byu/checkpoint/delaunay_graph_15_8_0.pth'
]

knn_conf = [0.1, 0.1, 0.4, 0.4, 0.4]
delaunay_conf = [0.4] * (len(delaunay_folds))

## **》》》 Ultralytics Offline Install**(v8.3.88[2025/03/11 ReleaseVersion])

In [ ]:
!pip install /kaggle/input/pip-install-pyg/torch_spline_conv-1.2.2+pt25cu124-cp310-cp310-linux_x86_64.whl
!pip install /kaggle/input/pip-install-pyg/torch_sparse-0.6.18+pt25cu124-cp310-cp310-linux_x86_64.whl
!pip install /kaggle/input/pip-install-pyg/pyg_lib-0.4.0+pt25cu124-cp310-cp310-linux_x86_64.whl
!pip install /kaggle/input/pip-install-pyg/torch_cluster-1.6.3+pt25cu124-cp310-cp310-linux_x86_64.whl
!pip install /kaggle/input/pip-install-pyg/torch_geometric-2.6.1-py3-none-any.whl

In [ ]:
"""[INFO]
* This notebookinstall Ultralytics v8.3.88(2025/03/11 ReleaseVersion)
  Can use YOLO12 is latest family version. 
* If you need a newer version, you can make it available by running and attaching the notebook.
  https://www.kaggle.com/code/hideyukizushi/ultralytics-offlineinstall-yolo12-weights
"""
!tar xfvz /kaggle/input/ultralytics-offlineinstall-yolo12-weights/archive.tar.gz
!pip install --no-index --find-links=./packages ultralytics
!rm -rf ./packages

## **》》》 Import Libs**

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import cv2
from tqdm.notebook import tqdm
from ultralytics import YOLO
import threading
import time
from contextlib import nullcontext
from concurrent.futures import ThreadPoolExecutor
import sys
import gc
sys.path.append('/kaggle/input/yolo-gnn-refine-byu')

In [ ]:
from torch_geometric.nn.models import GraphSAGE, GAT, CorrectAndSmooth
import torch.nn as nn
import torch
from torch_geometric.nn import LayerNorm
import torch.nn.functional as F
from torch_geometric.utils import dropout_edge
from torch.distributions import Beta
import torchvision.transforms as transforms
from torch_geometric.nn.conv import TransformerConv
from torch_geometric.data import Data
from torch_cluster import knn_graph
from torch_geometric.nn import radius_graph
from torch_geometric.transforms import AddRandomWalkPE
from scipy.spatial import Delaunay
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.neighbors import KDTree

## **》》》 Seed Fix**

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
data_path = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
test_dir = os.path.join(data_path, "test")
submission_path = "/kaggle/working/submission.csv"

## **》》》 GNN Refinement Tools**

In [ ]:
class DATA_CONFIG:
    radius = 30
    num_samples = 20
    thr = 10
    thr_sim = 0.25
    k = 15
    w = [0.5, 0.5]


class TRAIN_CONFIG:
    epochs = 200
    patience = 10
    batch_size = 8
    lr = 5e-4
    weight_decay = 0
    ckpt_epoch_freq = 1


class MODEL_CONIFG_1:
    num_layers = 8
    hidden_channels = 256
    pos_weight = 8
    jk = 'lstm'
    dropout = 0.3
    conf = 0.1
    graph_augment = False
    use_pe = True
    walk_length = 8
    weight_path = '/kaggle/input/yolo-gnn-refine-byu/checkpoint/knn_graph_15_8_4.pth'

class MODEL_CONIFG_2:
    num_layers = 8
    hidden_channels = 256
    pos_weight = 8
    jk = 'lstm'
    dropout = 0.3
    conf = 0.1
    graph_augment = False
    use_pe = True
    walk_length = 8
    weight_path = '/kaggle/input/yolo-gnn-refine-byu/checkpoint/delaunay_graph_15_8_4.pth'

In [ ]:
def get_feature_map(model):
    features = {}
    def make_hook(name):
        def hook(module, input, output):
            features[name] = output
        return hook
    model.model.model[16].register_forward_hook(make_hook("bifpn"))
    return features

In [ ]:
def delaunay_graph(x):
    points = x.cpu().numpy()  # assuming x is a tensor of shape [num_points, dims]
    tri = Delaunay(points)
    edges = set()

    for simplex in tri.simplices:
        for i in range(len(simplex)):
            for j in range(i + 1, len(simplex)):
                edges.add(tuple(sorted((simplex[i], simplex[j]))))

    edge_index = torch.tensor(list(edges), dtype=torch.long).t()
    return edge_index

def mst_graph(x):
    pos_np = x.numpy()
    dist_matrix = squareform(pdist(pos_np))  # pairwise distances
    mst = minimum_spanning_tree(dist_matrix)  # sparse MST
    row, col = mst.nonzero()
    edge_index = torch.tensor([row, col], dtype=torch.long)
    return edge_index


def kdtree_graph(x, k):
    N = x.shape[0]
    pos_np = x.numpy()
    tree = KDTree(pos_np)
    ind = tree.query(pos_np, k = k + 1, return_distance=False)  # k+1 because includes self
    # Convert neighbors to edge index
    row_idx = np.repeat(np.arange(N), k)
    col_idx = ind[:, 1:].reshape(-1)  # exclude self neighbor at index 0
    edge_index = torch.tensor([row_idx, col_idx], dtype=torch.long)
    return edge_index

In [ ]:
def assign_feat(x, feat, tomo_id, vol_size): # normalization back to feature's  dimension
    d, h, w = vol_size
    fd, fh, fw = feat.shape[0], feat.shape[2], feat.shape[3]
    z = x[:, 0].astype('int32')
    y = ((x[:, 1]/h) * fh).astype('int32')
    x = ((x[:, 2]/w) * fw).astype('int32')
    extract_feat = feat[z, :, y, x]
    return extract_feat

def sample_uniform_3d_ball(info, tomo_id, vol_size, radius=30, num_samples=20):
    """
    Uniformly sample points in 3D balls around each point without exceeding volume boundary.

    2. vec /= np.linalg.norm(vec, axis=1, keepdims=True)
    Normalizes each row (each vector) to unit length → now each vector lies on the unit sphere.
    Still shape: (n, 3)

    So now vec contains n unit direction vectors randomly distributed on the surface of the unit sphere.

    3. r = np.random.rand(n) ** (1/3)
    Generates n random radii from a uniform distribution inside a sphere.  To account for the fact that volume increases with r³, so we invert it for uniform volume density.
    Shape: (n,)

    4. r[:, None] (n,) → (n, 1)

    5. vec * (r[:, None] * radius) Multiplies each unit direction vector by a scaled radius.

    Shape: (n, 3) → each row is a 3D point.
        Args:
            points (np.ndarray): (N, 3) center points
            radius (float): radius of ball
            num_samples (int): number of samples per center
            volume_shape (tuple): (Z, Y, X) shape of 3D volume

    Returns:
        np.ndarray: (N, num_samples, 3) sampled points clipped to stay in volume
    """
    Z, Y, X = vol_size

    points = info[:, :-1] # get all the x y z
    N = points.shape[0] # numbner of points 

    def uniform_ball(n):
        vec = np.random.randn(n, 3)        #✅ Why keepdims=True? array of the norms (n,) → (n, 1)
        vec /= np.linalg.norm(vec, axis=1, keepdims=True) # “Compute the norm across columns (i.e., across the 3D components of each point).” => a shape (n,)  
                                # axis=0 = down the rows → column-wise ops
                                # axis=1 = across the columns → row-wise ops
        r = np.random.rand(n) ** (1/3) # → shape = (20, 1) 
        return vec * (r[:, None] * radius)  # (20, 3) = (20, 1) × (20, 3) element-wise scalar multiplication via broadcasting

    result = []
    np.random.seed(42)
    for center in points:
        attempts = 0
        accepted = []

        # Accept valid samples until we have enough or hit retry limit
        while len(accepted) < num_samples and attempts < num_samples * 10:
            samples = uniform_ball(num_samples)
            candidates = samples + center  # shifted samples

            # Keep only those within volume bounds
            mask = (
                (candidates[:, 0] >= 0) & (candidates[:, 0] < Z) &
                (candidates[:, 1] >= 0) & (candidates[:, 1] < Y) &
                (candidates[:, 2] >= 0) & (candidates[:, 2] < X)
            )
            accepted.extend(candidates[mask])
            attempts += 1

        # If not enough valid, pad with center point
        if len(accepted) < num_samples:
            accepted.extend([center] * (num_samples - len(accepted)))

        result.append(np.array(accepted[:num_samples]))

    result = np.concatenate(result, axis=0)
    return result


transform_ = AddRandomWalkPE(walk_length=MODEL_CONIFG_1.walk_length, attr_name=None)

def extract_tomo(info, feat, tomo_id, radius, num_samples):
    tomo_dir = os.path.join(test_dir, tomo_id)
    tomo_names = os.listdir(tomo_dir)
    D = len(tomo_names)
    img = cv2.imread(os.path.join(tomo_dir, tomo_names[0]))
    H, W = img.shape[:2]
    del img
    vol_size = [D, H, W]
    points = sample_uniform_3d_ball(info, tomo_id, vol_size, radius = 30, num_samples= 20) # ball randomized 
    extract_feat = assign_feat(points, feat, tomo_id, vol_size)   # normalization back to feature's  dimension
    #convert to torch
    points = torch.from_numpy(points)
    extract_feat = torch.from_numpy(extract_feat)
    batch = torch.zeros(points.shape[0], dtype=torch.int64)

    #multi-graph
    edge_index_1 = knn_graph(points, k=12, loop=False)
    # points_noisy = points + torch.randn_like(points) * 0.01  # small Gaussian noise
    # edge_index_2 = knn_graph(points_noisy, k=10, loop=False)
    edge_index_2 = knn_graph(points, k=12, loop=False)
    # tensor([
    # [10, 17, 14, 11,  3,  ...],  # Source node indices
    # [ 0,  0,  0,  1,  1,  ...]   # Target node indices

    
    edge_index_3 = delaunay_graph(points)
    data1= Data(points = points, x = extract_feat, edge_index = edge_index_1, batch = batch)
    data2= Data(points = points, x = extract_feat, edge_index = edge_index_2, batch = batch)
    data3= Data(points = points, x = extract_feat, edge_index = edge_index_3, batch = batch)

    data1 = transform_(data1)
    data2 = transform_(data2)
    data3 = transform_(data3)
    return data1, data2, data3

In [ ]:
class GraphModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.gnn = GraphSAGE(384 + cfg.walk_length if cfg.use_pe else 384,
                             num_layers= cfg.num_layers,
                             hidden_channels=cfg.hidden_channels,
                             out_channels=1,
                             jk=cfg.jk,
                             dropout=cfg.dropout,
                             norm=LayerNorm(cfg.hidden_channels),
                             )
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_index = edge_index.cuda()
        batch = batch.cuda()

        if edge_index.shape[0] == 0:
            edge_index = torch.tensor([[0, 0]]).T.cuda()

        x = x.cuda()
        logits = self.gnn(x, edge_index, batch=batch)
        return logits

## **》》》 Inference&Submission**

* GPU Init

In [ ]:
class GPUProfiler:
    def __init__(self, name):
        self.name = name
        self.start_time = None
        
    def __enter__(self):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.start_time = time.time()
        return self
        
    def __exit__(self, *args):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        elapsed = time.time() - self.start_time
        # print(f"[PROFILE] {self.name}: {elapsed:.3f}s")


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
if device.startswith('cuda'):
    # Set CUDA optimization flags
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cuda.matmul.allow_tf32 = True  # Allow TF32 on Ampere GPUs
    torch.backends.cudnn.allow_tf32 = True
    
    # Print GPU info
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert to GB
    print(f"Using GPU: {gpu_name} with {gpu_mem:.2f} GB memory")
    
    # Get available GPU memory and set batch size accordingly
    free_mem = gpu_mem - torch.cuda.memory_allocated(0) / 1e9
    BATCH_SIZE = max(8, min(32, int(free_mem * 4)))  # 4 images per GB as rough estimate
    print(f"Dynamic batch size set to {BATCH_SIZE} based on {free_mem:.2f}GB free memory")
else:
    print("GPU not available, using CPU")
    BATCH_SIZE = 4  # Reduce batch size for CPU

* Inference

In [ ]:
# import hdbscan
# from collections import defaultdict

In [ ]:
# #lower: accept noise
# #higher: discard noise 
# min_cluster_size = 2
# min_samples = 2

In [ ]:
# #hdbscan
# def postprocessing(points):
#     clusterer = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
#                                 min_samples = min_samples)
#     labels = clusterer.fit_predict(points)
#     clusters = defaultdict(list)
#     for label, point in zip(labels, points):
#         if label != -1: 
#             clusters[label].append(point)
#     voted_keypoints = [
#         tuple(np.mean(points, axis=0))
#         for points in clusters.values()
#     ]
#     cluster_size = [len(points) for points in clusters.values()]
#     print(cluster_size)
#     idx = np.argmax(cluster_size)
#     return voted_keypoints[idx]

In [ ]:
def normalize_slice(slice_data):
    """
    Normalize slice data using 2nd and 98th percentiles for better contrast
    """
    p2 = np.percentile(slice_data, 2)
    p98 = np.percentile(slice_data, 98)
    clipped_data = np.clip(slice_data, p2, p98)
    normalized = 255 * (clipped_data - p2) / (p98 - p2)
    return np.uint8(normalized)

def preload_image_batch(file_paths):
    """Preload a batch of images to CPU memory"""
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        if img is None:
            # Try with PIL as fallback
            img = np.array(Image.open(path))
        images.append(img)
    return images

def process_tomogram(tomo_id, model, model_gnns_1, model_gnns_2, index=0, total=1):
    """
    Process a single tomogram and return the most confident motor detection
    """
    # print(f"Processing tomogram {tomo_id} ({index}/{total})")
    
    # Get all slice files for this tomogram
    tomo_dir = os.path.join(test_dir, tomo_id)
    slice_files = sorted([f for f in os.listdir(tomo_dir) if f.endswith('.jpg')])
    
    # Apply CONCENTRATION to reduce the number of slices processed
    # This will process approximately CONCENTRATION fraction of all slices
    selected_indices = np.linspace(0, len(slice_files)-1, int(len(slice_files) * CONCENTRATION))
    selected_indices = np.round(selected_indices).astype(int)
    slice_files = [slice_files[i] for i in selected_indices]
    

    all_detections = []
    all_detections_tmp = []
    all_features_bifpn = []
    if device.startswith('cuda'):
        streams = [torch.cuda.Stream() for _ in range(min(4, BATCH_SIZE))]
    else:
        streams = [None]
    
    # Variables for preloading
    next_batch_thread = None
    next_batch_images = None
    
    # Process slices in batches
    for batch_start in tqdm(range(0, len(slice_files), BATCH_SIZE)):
        # Wait for previous preload thread if it exists
        if next_batch_thread is not None:
            next_batch_thread.join()
            next_batch_images = None
            
        batch_end = min(batch_start + BATCH_SIZE, len(slice_files))
        batch_files = slice_files[batch_start:batch_end]
        
        # Start preloading next batch
        next_batch_start = batch_end
        next_batch_end = min(next_batch_start + BATCH_SIZE, len(slice_files))
        next_batch_files = slice_files[next_batch_start:next_batch_end] if next_batch_start < len(slice_files) else []
        
        if next_batch_files:
            next_batch_paths = [os.path.join(tomo_dir, f) for f in next_batch_files]
            next_batch_thread = threading.Thread(target=preload_image_batch, args=(next_batch_paths,))
            next_batch_thread.start()
        else:
            next_batch_thread = None
        
        # Split batch across streams for parallel processing
        sub_batches = np.array_split(batch_files, len(streams))
        sub_batch_results = []
        
        for i, sub_batch in enumerate(sub_batches):
            if len(sub_batch) == 0:
                continue
                
            stream = streams[i % len(streams)]
            with torch.cuda.stream(stream) if stream and device.startswith('cuda') else nullcontext():
                # Process sub-batch
                sub_batch_paths = [os.path.join(tomo_dir, slice_file) for slice_file in sub_batch]
                sub_batch_slice_nums = [int(slice_file.split('_')[1].split('.')[0]) for slice_file in sub_batch]
                
                # Run inference with profiling
                with GPUProfiler(f"Inference batch {i+1}/{len(sub_batches)}"):
                    with torch.no_grad():
                        features = get_feature_map(model)
                        sub_results = model.predict(sub_batch_paths, imgsz=960, verbose=False)
                bifpn_feat = features['bifpn'].cpu().numpy()
                all_features_bifpn.append(bifpn_feat)
                del bifpn_feat
                
                # Process each result in this sub-batch
                for j, result in enumerate(sub_results):
                    if len(result.boxes) > 0:
                        boxes = result.boxes
                        for box_idx, confidence in enumerate(boxes.conf):
                            if confidence >= GNN_THR:
                                # Get bounding box coordinates
                                x1, y1, x2, y2 = boxes.xyxy[box_idx].cpu().numpy()
                                
                                # Calculate center coordinates
                                x_center = (x1 + x2) / 2
                                y_center = (y1 + y2) / 2

                                all_detections_tmp.append([round(sub_batch_slice_nums[j]),
                                                           round(y_center),
                                                           round(x_center),
                                                           float(confidence)
                                                          ])
                                
                                if confidence >= YOLO_THR:
                                    all_detections.append({
                                        'z': round(sub_batch_slice_nums[j]),
                                        'y': round(y_center),
                                        'x': round(x_center),
                                        'confidence': float(confidence)
                                    })
        
        # Synchronize streams
        if device.startswith('cuda'):
            torch.cuda.synchronize()
    
    # Clean up thread if still running
    if next_batch_thread is not None:
        next_batch_thread.join()

    if len(all_detections)==0:
        return {
            'tomo_id': tomo_id,
            'Motor axis 0': -1,
            'Motor axis 1': -1,
            'Motor axis 2': -1
        }

    all_detections_tmp = np.array(all_detections_tmp)
    all_features_bifpn = np.concatenate(all_features_bifpn, axis=0)
    data1, data2, data3 = extract_tomo(all_detections_tmp, all_features_bifpn, tomo_id, 30, 20) # get GNN train model                   # Data(points = points, x = extract_feat, edge_index = edge_index_1, batch = batch)
                                                                                                # ball randomized                       # BaseData, FeatureStore, GraphStore
                                                                                                # extract_feat = assign_feat(points, feat, tomo_id, vol_size)   # normalization back to feature's  dimension
    del all_detections_tmp
    del all_features_bifpn
    gc.collect()

    def conf_filter(pred, conf):
        return torch.where(pred>conf, pred, 0)

    #forward
    gnn_pred_1 = torch.stack(
        [conf_filter(model_gnns_1[0](data1).sigmoid()[:, 0].cpu(), knn_conf[0])] +\
        [conf_filter(_model_gnns_1(data2).sigmoid()[:, 0].cpu(), knn_conf[i+1]) for i, _model_gnns_1 in enumerate(model_gnns_1[1:])], dim=-1) #(N, 2)
    gnn_pred_1, _ = torch.max(gnn_pred_1, dim=-1)
    
    gnn_pred_2 = torch.stack(
        [conf_filter(model_gnn(data3).sigmoid()[:, 0].cpu(), delaunay_conf[i]) for i, model_gnn in enumerate(model_gnns_2)], dim=-1)
    gnn_pred_2, _ = torch.max(gnn_pred_2, dim=-1)
    
    gnn_pred = torch.cat([gnn_pred_1[gnn_pred_1>MODEL_CONIFG_1.conf],
                          gnn_pred_2[gnn_pred_2>MODEL_CONIFG_2.conf]], dim=0).cpu()

    try:
        points_1 = data1.points[gnn_pred_1>MODEL_CONIFG_1.conf]
        points_2 = data2.points[gnn_pred_2>MODEL_CONIFG_2.conf] + 1e-4
        points = torch.cat([points_1, points_2], dim=0)
        pred_idx = gnn_pred.argmax()
        point_pred = points[pred_idx].numpy() #(3, )
        # point_pred = postprocessing(points)
        return {
            'tomo_id': tomo_id,
            'Motor axis 0': point_pred[0],
            'Motor axis 1': point_pred[1],
            'Motor axis 2': point_pred[2]
        }
    except:
        return {
            'tomo_id': tomo_id,
            'Motor axis 0': -1,
            'Motor axis 1': -1,
            'Motor axis 2': -1
        }
    

def debug_image_loading(tomo_id):
    """
    Debug function to check image loading
    """
    tomo_dir = os.path.join(test_dir, tomo_id)
    slice_files = sorted([f for f in os.listdir(tomo_dir) if f.endswith('.jpg')])
    
    if not slice_files:
        print(f"No image files found in {tomo_dir}")
        return
        
    sample_file = slice_files[len(slice_files)//2]  # Middle slice
    img_path = os.path.join(tomo_dir, sample_file)
    
    # Try different loading methods
    try:
        # Method 1: PIL
        img_pil = Image.open(img_path)
        img_array_pil = np.array(img_pil)
        
        # Method 2: OpenCV
        img_cv2 = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        # print(f"OpenCV Image shape: {img_cv2.shape}, dtype: {img_cv2.dtype}")
        
        # Method 3: Convert to RGB
        img_rgb = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        
    # Also test with YOLO's built-in loader
    try:
        test_model = YOLO(model_path)
        test_results = test_model([img_path], verbose=False)
        # print("YOLO model successfully processed the test image")
    except Exception as e:
        print(f"Error with YOLO processing: {e}")

def load_kfold_models(cfg, fold_paths):
    models = []
    for path in fold_paths:
        model_gnn = GraphModel(cfg)
        model_gnn.load_state_dict(torch.load(path))
        model_gnn.to(device)
        model_gnn.eval()
        models.append(model_gnn)
    return models

def generate_submission():
    """
    Main function to generate the submission file
    """
    test_tomos = sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))])
    total_tomos = len(test_tomos)
    
    if test_tomos:
        debug_image_loading(test_tomos[0])
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print('load yolo')
    model = YOLO(model_path)
    model.to(device)

    print('load gnn')
    model_gnns_1 = load_kfold_models(MODEL_CONIFG_1, knn_folds)
    model_gnns_2 = load_kfold_models(MODEL_CONIFG_2, delaunay_folds)

    print('start inference')
    # Additional optimizations for inference
    if device.startswith('cuda'):
        # Fuse conv and bn layers for faster inference
        model.fuse()
        
        # Enable model half precision (FP16) if on compatible GPU
        if torch.cuda.get_device_capability(0)[0] >= 7:  # Volta or newer
            model.model.half()
    
    # Process tomograms with parallelization
    results = []
    motors_found = 0

    with ThreadPoolExecutor(max_workers=1) as executor:
        future_to_tomo = {}
        
        # Submit all tomograms for processing
        for i, tomo_id in enumerate(test_tomos, 1):
            future = executor.submit(process_tomogram, tomo_id, model, model_gnns_1, model_gnns_2, i, total_tomos)
            future_to_tomo[future] = tomo_id
        
        # Process completed futures as they complete
        for future in future_to_tomo:
            tomo_id = future_to_tomo[future]
            
            try:
                # Clear CUDA cache between tomograms
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    
                result = future.result()
                results.append(result)
                
                # Update motors found count
                has_motor = not pd.isna(result['Motor axis 0'])
                if has_motor:
                    motors_found += 1
                    print(f"Motor found in {tomo_id} at position: "
                          f"z={result['Motor axis 0']}, y={result['Motor axis 1']}, x={result['Motor axis 2']}")
                else:
                    print(f"No motor detected in {tomo_id}")
                    
                print(f"Current detection rate: {motors_found}/{len(results)} ({motors_found/len(results)*100:.1f}%)")
            
            except Exception as e:
                print(f"Error processing {tomo_id}: {e}")
                # Create a default entry for failed tomograms
                results.append({
                    'tomo_id': tomo_id,
                    'Motor axis 0': -1,
                    'Motor axis 1': -1,
                    'Motor axis 2': -1
                })
    
    # Create submission dataframe
    submission_df = pd.DataFrame(results)
    
    # Ensure proper column order
    submission_df = submission_df[['tomo_id', 'Motor axis 0', 'Motor axis 1', 'Motor axis 2']]
    
    # Save the submission file
    submission_df.to_csv(submission_path, index=False)
    print("="*50)
    print("= Submission preview:")
    print("="*50)
    print(submission_df.head())
    
    return submission_df

In [ ]:
submission = generate_submission()